In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error

In [2]:
X_train = pd.read_csv('Train.csv')
X_test = pd.read_csv('Test.csv')
otvet = X_test[['id']]
y_train = X_train['price']
X_train = X_train.drop('price', 1)
X_train = X_train.replace(np.nan, -999)
X_test = X_test.replace(np.nan, -999)
X_train = X_train.drop('date', 1)
X_test = X_test.drop('date', 1)
X_train = X_train.drop('id', 1)
X_test = X_test.drop('id', 1)

In [3]:
from sklearn.ensemble import GradientBoostingRegressor
mdl = GradientBoostingRegressor(random_state=1, learning_rate=1, loss='huber', n_estimators=1000)
mdl.fit(X_train, y_train)
y_pred = mdl.predict(X_test)

In [4]:
otvet['price'] = y_pred
otvet.to_csv('output.csv', index=False)
otvet

,id,price
0,100000,4.320934e+06
1,100001,9.124757e+06
2,100002,2.980955e+06
3,100003,7.679209e+06
4,100004,1.364590e+06
...,...,...
99995,199995,4.944580e+06
99996,199996,3.918351e+06
99997,199997,7.347274e+06
99998,199998,3.349533e+06
